In [19]:
import numpy as np
from IPython.display import display, Markdown
from meowmeow import (
    Mat, Vec, display_latex, display_math, _err,
    _as_array_M, _as_array_v,
    forward_subst, backward_subst,
    pre_assess, make_random_system_simple,
    gauss_ops_theory, gauss_solve, _read_square_A_and_b, _parse_pivot
)

In [3]:
def _ops_zero():
    return {"add":0,"sub":0,"mul":0,"div":0,"sqrt":0,"total":0}

In [4]:
def _ops_inc(ops, key, k=1):
    ops[key] += k; ops["total"] += k

In [5]:
def _ops_merge(*many):
    out = _ops_zero()
    for d in many:
        for k in out:
            out[k] += d.get(k, 0)
    return out

In [6]:
def cholesky_ops_theory(n: int):
    return {"fact": (1/3)*n**3, "tri": 2*n**2, "total": (1/3)*n**3 + 2*n**2}

In [7]:
def cholesky(A_in, log_every=None, checkpoints=None):
    A = _as_array_M(A_in).astype(float, copy=True)
    n = A.shape[0]
    if not np.allclose(A, A.T, atol=1e-12, rtol=1e-12):
        raise _err("Холецкий требует симметричности: A ≠ A^T")

    L = np.zeros_like(A)
    ops = _ops_zero()

    def _maybe(k):
        if log_every and log_every > 0 and k % log_every == 0:
            display(Markdown(f"**Cholesky: шаг k = {k}**"))
            display_latex(Mat(L), label=r"L")

    _maybe(0)
    for k in range(n):
        s = 0.0
        for j in range(k):
            s += L[k, j] * L[k, j]; _ops_inc(ops, "mul"); _ops_inc(ops, "add")
        val = A[k, k] - s; _ops_inc(ops, "sub")
        if not np.isfinite(val) or val <= 0.0:
            raise _err("не спо: на диагонали получилось неположительное значение")
        L[k, k] = np.sqrt(val); _ops_inc(ops, "sqrt")

        for i in range(k+1, n):
            s = 0.0
            for j in range(k):
                s += L[i, j] * L[k, j]; _ops_inc(ops, "mul"); _ops_inc(ops, "add")
            L[i, k] = (A[i, k] - s) / L[k, k]; _ops_inc(ops, "sub"); _ops_inc(ops, "div")
        _maybe(k+1)

    return Mat(L), ops

In [8]:
def solve_via_cholesky(L_in, b_in):
    L = _as_array_M(L_in)
    b = _as_array_v(b_in)
    y, of = forward_subst(L, b)
    x, ob = backward_subst(L.T, y)
    return Vec(x), _ops_merge(of, ob)

In [9]:
def make_non_spd(n: int, seed: int | None = 7):
    rng = np.random.default_rng(seed)
    G = rng.standard_normal((n, n))
    S = (G + G.T) / 2.0
    S -= (abs(np.linalg.eigvalsh(S)).max() + 1e-3) * np.eye(n)
    return Mat(S)

In [21]:
def theory_note(kind: str):
    if kind == "spd":
        display(Markdown(
            r"все главные миноры $>0 \;\Rightarrow\;$ существует факторизация $A = LL^\top$"
        ))
    elif kind == "nonspd_sym":
        display(Markdown(
            r"на шаге $l_{kk}^2 = a_{kk} - \sum_j l_{kj}^2$ остаётся $\le 0$"
        ))
    elif kind == "nonsym":
        display(Markdown(
            r"Холецкий неприменим: $A \ne A^\top$"
        ))

In [42]:
def _print_metrics(A, b, x_hat):
    A_arr = _as_array_M(A); b_arr = _as_array_v(b); x_arr = x_hat.data
    r = A_arr @ x_arr - b_arr
    rel_res = np.linalg.norm(r, 2) / (np.linalg.norm(A_arr, 2) * np.linalg.norm(x_arr, 2))
    display(Markdown(
    fr"$\mathrm{{rel\_res}}=\frac{{\lVert A x-b\rVert_2}}{{\lVert A\rVert_2\,\lVert x\rVert_2}}"
    fr"=\frac{{{np.linalg.norm(r,2):.6g}}}{{{np.linalg.norm(A_arr,2):.6g}\cdot{np.linalg.norm(x_arr,2):.6g}}}"
    fr"={rel_res:.6g}$"))

In [ ]:
def _compare_ops_with_gauss(A, b, ops_chol_fact, ops_chol_tri):
    total_chol = _ops_merge(ops_chol_fact, ops_chol_tri)

    pivot = _parse_pivot(input("главный элемент для Гаусса? (`none`/`col`/`row`/`full`, по умолчанию col): "))
    xg, info = gauss_solve(A, b, pivot=pivot)

    display(Markdown("**сравнение числа операций**"))
    display(Markdown("**Холецкий:**"))
    display(Markdown(f"разложение (2): `Q = {ops_chol_fact['total']}`"))
    display(Markdown(f"решение систем (4): `Q = {ops_chol_tri['total']}`"))
    display(Markdown(f"всего: `Q = {total_chol['total']}`"))
    display(Markdown("**Гаусс:**"))
    display(Markdown(f"всего: `Q = {info['ops']['total']}`"))

    n = _as_array_M(A).shape[0]
    th_g = gauss_ops_theory(n)
    th_c = cholesky_ops_theory(n)
    display(Markdown("**теоретические разности:**"))
    display(Markdown(f"Гаусс (одна СЛАУ): `~ {th_g['total']:.0f}`"))
    display(Markdown(f"Холецкий (одна СЛАУ): `~ {th_c['total']:.0f}`"))

    A_arr = _as_array_M(A); b_arr = _as_array_v(b)
    rg = A_arr @ xg.data - b_arr
    rel_res_g = np.linalg.norm(rg, 2) / (np.linalg.norm(A_arr, 2) * np.linalg.norm(xg.data, 2))
    A_arr = _as_array_M(A); b_arr = _as_array_v(b)
    rg = A_arr @ xg.data - b_arr

    num = float(np.linalg.norm(rg, 2))
    normA = float(np.linalg.norm(A_arr, 2))
    normx = float(np.linalg.norm(xg.data, 2))
    rel_res_g = num / (normA * normx)

    tex = (
        fr"$\mathrm{{rel\_res}}^{{(\mathrm{{gauss}})}}"
        fr"=\dfrac{{\lVert A\,\hat x^{{(\mathrm{{gauss}})}}-b\rVert_2}}"
        fr"{{\lVert A\rVert_2\,\lVert \hat x^{{(\mathrm{{gauss}})}}\rVert_2}}"
        fr"=\dfrac{{{num:.6g}}}{{{normA:.6g}\cdot{normx:.6g}}}"
        fr"={rel_res_g:.6g}$"
    )
    display(Markdown(tex))


In [38]:
def main():
    display(Markdown(
        "[1] решить $A x=b$ (ручной ввод / случайная симметричная положительно определённая (спо) / не спо)\n\n"
        "[2] глазеем сразу на три случая (симметричная / симм. не положительно определённая / несимметричная)\n"
    ))
    mode = input("ваш выбор (1/2): ").strip()

    if mode == "1":
        how = (input("как задаём? (m=вручную / r=случайная спо / bad=не спо / ns=несимметричная): ").strip().lower() or "m")
        
        if how == "m":
            A, b = _read_square_A_and_b()
            x_true = None
        elif how == "r":
            n = int(input("n = ").strip() or "5")
            A, x_true, b = make_random_system_simple(n, spd=True, show=True)
        elif how == "bad":
            n = int(input("n = ").strip() or "5")
            A = make_non_spd(n)
            _, x_true, b = make_random_system_simple(n, spd=False, show=False)
            display(Markdown("сгенерирована симметричная, но не положительно определённая матрица"))
            theory_note("nonspd_sym")
        elif how == "ns":
            n = int(input("n = ").strip() or "5")
            rng = np.random.default_rng(123)
            A = Mat(rng.standard_normal((n, n)))
            _, x_true, b = make_random_system_simple(n, spd=False, show=False)
            display(Markdown("сгенерирована несимметричная матрица"))
            theory_note("nonsym")
        else:
            print("мяу, не узнали режим :("); return

        stats = pre_assess(A)
        display(Markdown(r"всякое:"))
        display_latex(A, label=r"A")
        display_latex(stats["cond2"], label=r"\mathrm{cond}_2(A)")
        display_latex(stats["det"], label=r"\det(A)")

        try:
            L, ops_fact = cholesky(A)
            x_chol, ops_tri = solve_via_cholesky(L, b)
            do_cmp = (input("сравнить число операций с Гауссом? (y/n, по умолчанию y): ").strip().lower() or "y").startswith("y")
            if do_cmp:
                _compare_ops_with_gauss(A, b, ops_fact, ops_tri)

        except Exception as e:
            display(Markdown(f"**Холецкий не сработал:** `{e}`"))

        print("мяу, готово (=^..^=)")
        return

    if mode == "2":
        n = int(input("n = ").strip() or "5")

        display(Markdown("**1. симметричная положительно определённая (решаемо)**"))
        A1, x_true1, b1 = make_random_system_simple(n, spd=True, show=True)
        display_latex(A1, label=r"A^{(1)}")
        theory_note("spd")
        L1, _ = cholesky(A1)
        x1, _ = solve_via_cholesky(L1, b1)
        display_latex(x1, label=r"x^{(1)}_{\mathrm{chol}}")
        _print_metrics(A1, b1, x1)

        display(Markdown("**2. симметричная, но не положительно определённая (не решаемо Холецким)**"))
        A2 = make_non_spd(n)
        display_latex(A2, label=r"A^{(2)}")
        _, x_true2, b2 = make_random_system_simple(n, spd=False, show=False)
        theory_note("nonspd_sym")
        try:
            L2, _ = cholesky(A2)
            display_latex(L2, label=r"L^{(2)}")
        except Exception as e:
            display(Markdown(f"ожидаемо упало: `{e}`"))

        display(Markdown("**3. несимметричная (тоже не решаемо Холецким)**"))
        rng = np.random.default_rng(321)
        A3 = Mat(rng.standard_normal((n, n)))
        display_latex(A3, label=r"A^{(3)}")
        _, x_true3, b3 = make_random_system_simple(n, spd=False, show=False)
        theory_note("nonsym")
        try:
            L3, _ = cholesky(A3)
            display_latex(L3, label=r"L^{(3)}")
        except Exception as e:
            display(Markdown(f"ожидаемо упало: `{e}`"))

        print("мяу, готово (=^..^=)")
        return

    print("мяу, нужно выбрать 1 или 2")


In [45]:
main()

[1] решить $A x=b$ (ручной ввод / случайная симметричная положительно определённая (спо) / не спо)

[2] глазеем сразу на три случая (симметричная / симм. не положительно определённая / несимметричная)


**1. симметричная положительно определённая (решаемо)**

случайная матрица $A$, симметричная положительно определённая; сгенерирован точный $x_{\rm true}\sim\mathcal U[-10.0,10.0]$ (seed=42):

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

**3. несимметричная (тоже не решаемо Холецким)**

<IPython.core.display.Math object>

Холецкий неприменим: $A \ne A^\top$

ожидаемо упало: `мяу мяу, Холецкий требует симметричности: A ≠ A^T`

мяу, готово (=^..^=)
